## Imports

In [1]:
import os
path = "/Users/patricia/Documents/code/python-code/behavior-detection/src"
os.chdir(path)  # Muda o diretório para o nível anterior (a raiz do projeto)
print(os.getcwd())  # Verifique se agora está na raiz

/Users/patricia/Documents/code/python-code/behavior-detection/src


In [2]:
from sklearn import datasets
from time import time
import pandas as pd

## Data

In [3]:
digits = datasets.load_digits()

n_samples = len(digits.images)
X = digits.images
y = digits.target

# Flatten the images
X = X.reshape((n_samples, -1))
X = pd.DataFrame(X)

In [4]:
print(n_samples)
print(type(X))
print(X.head(5))
print(y[:5])

1797
<class 'pandas.core.frame.DataFrame'>
    0    1    2     3     4     5    6    7    8    9   ...   54   55   56  \
0  0.0  0.0  5.0  13.0   9.0   1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1  0.0  0.0  0.0  12.0  13.0   5.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2  0.0  0.0  0.0   4.0  15.0  12.0  0.0  0.0  0.0  0.0  ...  5.0  0.0  0.0   
3  0.0  0.0  7.0  15.0  13.0   1.0  0.0  0.0  0.0  8.0  ...  9.0  0.0  0.0   
4  0.0  0.0  0.0   1.0  11.0   0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

    57   58    59    60    61   62   63  
0  0.0  6.0  13.0  10.0   0.0  0.0  0.0  
1  0.0  0.0  11.0  16.0  10.0  0.0  0.0  
2  0.0  0.0   3.0  11.0  16.0  9.0  0.0  
3  0.0  7.0  13.0  13.0   9.0  0.0  0.0  
4  0.0  0.0   2.0  16.0   4.0  0.0  0.0  

[5 rows x 64 columns]
[0 1 2 3 4]


In [5]:
from sklearn.model_selection import train_test_split

# Dividir X e y em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Preprocessing

In [6]:
import pandas as pd
from core.preprocessors.data_encoder import DataEncoder

# Codificar y_train
y_train = DataEncoder.encode_y(y_train)
# Codificar y_test
y_test = DataEncoder.encode_y(y_test)

# Pré-processar X_train
X_encoder = DataEncoder(num_classes=0, select_numerical=True)
X_encoder.fit(X_train)

X_train = X_encoder.transform(X_train)

# Pré-processar X_test usando o mesmo preprocessor
X_test = X_encoder.transform(X_test)

In [7]:
print(X_train.head(15))

      num_standard__x0  num_standard__x1  num_standard__x2  num_standard__x3  \
1734               0.0         -0.341698         -0.463360          0.508365   
855                0.0         -0.341698          0.784716         -0.658600   
1642               0.0         -0.341698         -1.087399         -0.425207   
175                0.0          0.759243          0.992729          0.975151   
925                0.0         -0.341698          0.160678          0.508365   
548                0.0         -0.341698          1.616768          0.975151   
1615               0.0         -0.341698         -1.087399         -0.658600   
334                0.0         -0.341698         -0.463360         -0.191814   
756                0.0         -0.341698         -1.087399         -0.658600   
433                0.0         -0.341698         -1.087399         -1.592172   
889                0.0          5.163004          2.240806          0.041579   
1675               0.0          0.759243

## Classifier

In [8]:
# Definir quais modelos e seletores utilizar
selected_models = [ # None to use all models
    # 'Logistic Regression',
    # 'Decision Tree',
    'Random Forest',
    # 'Gradient Boosting',
    # 'SVM',
    # 'KNN',
    # 'XGBoost'
]
selected_selectors = ['pca']

cv = 5  # Number of folds in the cross-validation
n_iter = 100
n_jobs = 4  # Number of processors to be used in the execution: -1 to use all processors

# Choose a scoring metric
scoring_metric = 'balanced_accuracy'  # Possible values: 'f1_macro', 'balanced_accuracy', 'roc_auc_ovr', etc.

## Usando Otimização Bayesiana (BayesSearchCV)

In [9]:
from core.training.skopt_bayesian_optimization_training import SkoptBayesianOptimizationTraining

training = SkoptBayesianOptimizationTraining()

#### Executar o treinamento
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

#### Exemplo de acesso aos modelos treinados
for model_key, model_info in trained_models.items():
    print(f"Modelo: {model_key}")
    print(f"Melhores Hiperparâmetros: {model_info['hyperparameters']}")
    print(f"Resultado CV: {model_info['cv_result']}\n") 

INFO:root:Training and evaluating Random Forest with Bayesian Optimization and pca
INFO:root:Search space for Random Forest with selector pca: {'classifier__n_estimators': Integer(low=50, high=300, prior='uniform', transform='identity'), 'classifier__max_depth': Integer(low=3, high=30, prior='uniform', transform='identity'), 'classifier__min_samples_split': Integer(low=2, high=20, prior='uniform', transform='identity'), 'classifier__min_samples_leaf': Integer(low=1, high=10, prior='uniform', transform='identity'), 'classifier__max_features': Categorical(categories=('sqrt', 'log2', None), prior=None), 'feature_selection__n_components': [1, 2, 3, 4, 5]}


Training and evaluating Random Forest with Bayesian Optimization and pca
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END classifier__max_depth=14, classifier__max_features=None, classifier__min_samples_leaf=9, classifier__min_samples_split=8, classifier__n_estimators=218, feature_selection__n_components=3;, score=0.708 total time=   0.4s
[CV 2/5] END classifier__max_depth=14, classifier__max_features=None, classifier__min_samples_leaf=9, classifier__min_samples_split=8, classifier__n_estimators=218, feature_selection__n_components=3;, score=0.755 total time=   0.4s
[CV 3/5] END classifier__max_depth=14, classifier__max_features=None, classifier__min_samples_leaf=9, classifier__min_samples_split=8, classifier__n_estimators=218, feature_selection__n_components=3;, score=0.756 total time=   0.4s
[CV 4/5] END classifier__max_depth=14, classifier__max_features=None, classifier__min_samples_leaf=9, classifier__min_samples_split=8, classifier__n_estimators=218, feature

INFO:root:Iteration 1: tested parameters: [14, None, 9, 8, 218, 3], score: 0.7316827044746956


[CV 5/5] END classifier__max_depth=14, classifier__max_features=None, classifier__min_samples_leaf=9, classifier__min_samples_split=8, classifier__n_estimators=218, feature_selection__n_components=3;, score=0.697 total time=   0.3s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END classifier__max_depth=26, classifier__max_features=None, classifier__min_samples_leaf=4, classifier__min_samples_split=19, classifier__n_estimators=266, feature_selection__n_components=1;, score=0.345 total time=   0.3s
[CV 2/5] END classifier__max_depth=26, classifier__max_features=None, classifier__min_samples_leaf=4, classifier__min_samples_split=19, classifier__n_estimators=266, feature_selection__n_components=1;, score=0.338 total time=   0.3s
[CV 3/5] END classifier__max_depth=26, classifier__max_features=None, classifier__min_samples_leaf=4, classifier__min_samples_split=19, classifier__n_estimators=266, feature_selection__n_components=1;, score=0.336 total time=   0.3s
[CV 4/5] E

INFO:root:Iteration 2: tested parameters: [26, None, 4, 19, 266, 1], score: 0.3449824378645291


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 4/5] END classifier__max_depth=12, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=300, feature_selection__n_components=4;, score=0.853 total time=   0.6s
[CV 3/5] END classifier__max_depth=12, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=300, feature_selection__n_components=4;, score=0.817 total time=   0.6s
[CV 2/5] END classifier__max_depth=12, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=300, feature_selection__n_components=4;, score=0.841 total time=   0.6s
[CV 1/5] END classifier__max_depth=12, classifier__max_features=sqrt, classifier__min_samples_leaf=1, classifier__min_samples_split=2, classifier__n_estimators=300, feature_selection__n_components=4;, score=0.808 total time=   0.7s
[CV 5/5] END 

INFO:root:Iteration 99: tested parameters: [12, 'sqrt', 1, 2, 300, 4], score: 0.8207464790390263


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END classifier__max_depth=22, classifier__max_features=None, classifier__min_samples_leaf=1, classifier__min_samples_split=19, classifier__n_estimators=286, feature_selection__n_components=4;, score=0.794 total time=   0.8s
[CV 3/5] END classifier__max_depth=22, classifier__max_features=None, classifier__min_samples_leaf=1, classifier__min_samples_split=19, classifier__n_estimators=286, feature_selection__n_components=4;, score=0.821 total time=   0.8s
[CV 2/5] END classifier__max_depth=22, classifier__max_features=None, classifier__min_samples_leaf=1, classifier__min_samples_split=19, classifier__n_estimators=286, feature_selection__n_components=4;, score=0.809 total time=   0.8s
[CV 4/5] END classifier__max_depth=22, classifier__max_features=None, classifier__min_samples_leaf=1, classifier__min_samples_split=19, classifier__n_estimators=286, feature_selection__n_components=4;, score=0.829 total time=   0.8s
[CV 5/5] 

INFO:root:Iteration 100: tested parameters: [22, None, 1, 19, 286, 4], score: 0.8042051073797459
2024-10-20 17:53:56,745 | skopt_training | INFO | Best parameters: OrderedDict([('classifier__max_depth', 30), ('classifier__max_features', 'sqrt'), ('classifier__min_samples_leaf', 1), ('classifier__min_samples_split', 2), ('classifier__n_estimators', 300), ('feature_selection__n_components', 5)])
INFO:skopt_training:Best parameters: OrderedDict([('classifier__max_depth', 30), ('classifier__max_features', 'sqrt'), ('classifier__min_samples_leaf', 1), ('classifier__min_samples_split', 2), ('classifier__n_estimators', 300), ('feature_selection__n_components', 5)])
2024-10-20 17:53:56,746 | skopt_training | INFO | Best cross-validation score: 0.8553389598083703
INFO:skopt_training:Best cross-validation score: 0.8553389598083703
2024-10-20 17:53:56,746 | skopt_training | INFO | All hyperparameter combinations and their cross-validation results:
INFO:skopt_training:All hyperparameter combinatio

Bayesian Optimization Best Result for Random Forest with pca: 0.8553389598083703
Modelo: Random Forest_pca
Melhores Hiperparâmetros: {'memory': None, 'steps': [('feature_selection', PCA(n_components=5)), ('classifier', RandomForestClassifier(max_depth=30, n_estimators=300))], 'verbose': False, 'feature_selection': PCA(n_components=5), 'classifier': RandomForestClassifier(max_depth=30, n_estimators=300), 'feature_selection__copy': True, 'feature_selection__iterated_power': 'auto', 'feature_selection__n_components': 5, 'feature_selection__n_oversamples': 10, 'feature_selection__power_iteration_normalizer': 'auto', 'feature_selection__random_state': None, 'feature_selection__svd_solver': 'auto', 'feature_selection__tol': 0.0, 'feature_selection__whiten': False, 'classifier__bootstrap': True, 'classifier__ccp_alpha': 0.0, 'classifier__class_weight': None, 'classifier__criterion': 'gini', 'classifier__max_depth': 30, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': None, 'c

## Usando Otimização Bayesiana (Optuna)

# Importação da nova classe OptunaBayesianOptimizationTraining
from core.training.optuna_bayesian_optimization_training import OptunaBayesianOptimizationTraining

# Instanciação da classe de treinamento com Otimização Bayesiana via Optuna
training = OptunaBayesianOptimizationTraining()

# Executar o treinamento
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,  # Será mapeado para n_trials na classe OptunaBayesianOptimizationTraining
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

# Exemplo de acesso aos modelos treinados
for model_key, model_info in trained_models.items():
    print(f"Modelo: {model_key}")
    print(f"Melhores Hiperparâmetros: {model_info['hyperparameters']}")
    print(f"Resultado CV: {model_info['cv_result']}\n")


## Using GridSearchCV

In [10]:
from core.training.grid_search_training import GridSearchTraining

# Instantiate the GridSearchCV training class
training = GridSearchTraining()

# Execute the training
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,  # This parameter is not used in GridSearchCV but kept for consistency
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

# Example of accessing the trained models
for model_key, model_info in trained_models.items():
    print(f"Model: {model_key}")
    print(f"Best Hyperparameters: {model_info['hyperparameters']}")
    print(f"CV Result: {model_info['cv_result']}\n")

2024-10-20 17:53:56,897 | grid_search_training | INFO | Training and evaluating Random Forest with GridSearchCV and pca
INFO:grid_search_training:Training and evaluating Random Forest with GridSearchCV and pca
DEBUG:grid_search_training:Parameter grid: {'classifier__n_estimators': [50, 100, 200], 'classifier__max_depth': [None, 10, 20, 30], 'classifier__min_samples_split': [2, 5, 10], 'classifier__min_samples_leaf': [1, 2, 4], 'feature_selection__n_components': [1, 2, 3, 4, 5]}
2024-10-20 17:53:56,898 | grid_search_training | INFO | Starting GridSearchCV fitting process
INFO:grid_search_training:Starting GridSearchCV fitting process


Fitting 5 folds for each of 540 candidates, totalling 2700 fits


2024-10-20 17:56:22,160 | grid_search_training | INFO | GridSearchCV fitting process completed
INFO:grid_search_training:GridSearchCV fitting process completed
2024-10-20 17:56:22,166 | grid_search_training | INFO | Best parameters: {'classifier__max_depth': 20, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100, 'feature_selection__n_components': 5}
INFO:grid_search_training:Best parameters: {'classifier__max_depth': 20, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100, 'feature_selection__n_components': 5}
2024-10-20 17:56:22,167 | grid_search_training | INFO | Best cross-validation score: 0.852708370852976
INFO:grid_search_training:Best cross-validation score: 0.852708370852976
2024-10-20 17:56:22,168 | grid_search_training | INFO | All hyperparameter combinations and their cross-validation results:
INFO:grid_search_training:All hyperparameter combinations and their cross-valida

Model: Random Forest_pca
Best Hyperparameters: {'classifier__max_depth': 20, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 100, 'feature_selection__n_components': 5}
CV Result: 0.852708370852976



### Treinando com RandomSearchCV

In [11]:
# src/notebooks/model_training_behavior_multiclassification_by_student_level.ipynb

from core.training.random_search_training import RandomSearchTraining

# Instantiate the RandomizedSearchCV training class
training = RandomSearchTraining()

# Execute the training
trained_models = training.train_model(
    X_train=X_train,
    y_train=y_train,
    selected_models=selected_models,
    selected_selectors=selected_selectors,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

# Example of accessing the trained models
for model_key, model_info in trained_models.items():
    print(f"Model: {model_key}")
    print(f"Best Hyperparameters: {model_info['hyperparameters']}")
    print(f"CV Result: {model_info['cv_result']}\n")

2024-10-20 17:56:22,971 | random_search_training | INFO | Training and evaluating Random Forest with RandomizedSearchCV and pca
INFO:random_search_training:Training and evaluating Random Forest with RandomizedSearchCV and pca
DEBUG:random_search_training:Parameter grid: {'classifier__n_estimators': [50, 100, 200], 'classifier__max_depth': [None, 10, 20, 30], 'classifier__min_samples_split': [2, 5, 10], 'classifier__min_samples_leaf': [1, 2, 4], 'feature_selection__n_components': [1, 2, 3, 4, 5]}
2024-10-20 17:56:22,972 | random_search_training | INFO | Starting RandomizedSearchCV fitting process
INFO:random_search_training:Starting RandomizedSearchCV fitting process


Fitting 5 folds for each of 100 candidates, totalling 500 fits


2024-10-20 17:56:53,633 | random_search_training | INFO | RandomizedSearchCV fitting process completed
INFO:random_search_training:RandomizedSearchCV fitting process completed
2024-10-20 17:56:53,634 | random_search_training | INFO | Best parameters: {'feature_selection__n_components': 5, 'classifier__n_estimators': 100, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 2, 'classifier__max_depth': None}
INFO:random_search_training:Best parameters: {'feature_selection__n_components': 5, 'classifier__n_estimators': 100, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 2, 'classifier__max_depth': None}
2024-10-20 17:56:53,635 | random_search_training | INFO | Best cross-validation score: 0.8532495365220616
INFO:random_search_training:Best cross-validation score: 0.8532495365220616
2024-10-20 17:56:53,636 | random_search_training | INFO | All hyperparameter combinations and their cross-validation results:
INFO:random_search_training:All hyperparameter

Model: Random Forest_pca
Best Hyperparameters: {'feature_selection__n_components': 5, 'classifier__n_estimators': 100, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 2, 'classifier__max_depth': None}
CV Result: 0.8532495365220616



# Avaliação dos Modelos

In [12]:
from core.evaluation.evaluation import Evaluation  

feature_names = X_train.columns  # Assumindo que os nomes das características são as colunas
class_metrics_results, avg_metrics_results = Evaluation.evaluate_all_models(trained_models, X_train, y_train, X_test, y_test, feature_names)

/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/projetos_ML/lib/python3.11/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


# Geração dos Relatórios

In [13]:
from core.logging.report_formatter import ReportFormatter
from core.logging.file_utils import FileUtils

directory = "../output/"

# Gerar relatório textual a partir dos resultados de avaliação
text_report = ReportFormatter.generate_text_report(class_metrics_results, avg_metrics_results)

# Imprimir ou salvar o relatório
FileUtils.save_file_with_timestamp(text_report, "general_report.txt", directory)

# Gerar DataFrame detalhado dos relatórios por classe
class_report_df = ReportFormatter.generate_class_report_dataframe(class_metrics_results)

# Gerar DataFrame resumido dos relatórios de métricas médias
avg_metrics_report_df = ReportFormatter.generate_avg_metrics_report_dataframe(avg_metrics_results)

# Salvar os DataFrames como arquivos CSV, se necessário
FileUtils.save_file_with_timestamp(class_report_df, "class_report.csv", directory, is_csv=True)
FileUtils.save_file_with_timestamp(avg_metrics_report_df, "avg_metrics_report.csv", directory, is_csv=True)


'../output/avg_metrics_report_20241020_1756.csv'

# Salvando os modelos em arquivos para recuperação

In [14]:
from core.logging.model_manager import ModelManager

# Caminhos
model_dir = "../models/"

# Salvar todos os modelos
saved_models = ModelManager.save_all_models(trained_models, model_dir)
print("Modelos salvos:", saved_models)

Model 'Random Forest_pca' saved at: ../models/model_Random Forest_pca_20241020_1756.pkl
Modelos salvos: ['../models/model_Random Forest_pca_20241020_1756.pkl']
